In [4]:
import spikeextractors as se
import spiketoolkit as st
import numpy as np
import matplotlib.pylab as plt
import exdir
import expipe
import scipy.signal as ss
import pyintan
import sys
sys.path.append('../ca2-mec')
import data_processing as dp

%matplotlib notebook

def find_fmax_noise(recording_hp, start_frame=0, end_frame=300000, start_freq=2000, end_freq=4000):
    import scipy.signal as ss
    filt_traces = recording_hp.get_traces(start_frame=start_frame, end_frame=end_frame)
    f, p = ss.welch(filt_traces, recording_hp.get_sampling_frequency())
    idxs = np.where((f > start_freq) & (f < end_freq))

    max_freq = f[idxs][np.squeeze(p[:, idxs]).mean(axis=0).argmax()]
    return max_freq

In [11]:
project_path = dp.project_path()
project = expipe.get_project(project_path)
actions = project.actions

SyntaxError: unexpected character after line continuation character (<ipython-input-11-e9c16b41181f>, line 1)

In [6]:
actions

NameError: name 'actions' is not defined

In [ ]:
action = actions['002-111220-1']

In [ ]:
check_hp = False
check_lfp = True

In [ ]:
data_path = dp.get_data_path(action)

In [ ]:
# check if openephys or intan
exdir_file = exdir.File(str(data_path))
acquisition = exdir_file['acquisition']

In [ ]:
attrs = acquisition.attrs

In [ ]:
if 'Intan' in attrs['acquisition_system']:
    print('Intan recording')
    prb_file = 'probes/tetrode_32_intan.prb'
    intan = True
    openephys = False
    rhs_file = [x for x in acquisition[attrs['session']].directory.iterdir() if x.suffix == '.rhs'][0]
    f = pyintan.File(str(rhs_file))
    dig_intan = f.digital_in_events
    recording = se.IntanRecordingExtractor(rhs_file)
elif 'FPGA' in attrs['acquisition_system']:
    print('Open Ephys Recording')
    prb_file = 'probes/tetrode_32_openephys.prb'
    openephys = True
    intan = False
    recording = se.OpenEphysRecordingExtractor(str(acquisition[attrs['session']].directory))

In [ ]:
#recording = se.load_probe_file(recording, prb_file)
#recording_f = st.preprocessing.bandpass_filter(recording, type='butter', order=5)
#recording_cmr = st.preprocessing.common_reference(recording_f, reference='median', groups=[list(range(16)), list(range(16,32))])

#if intan:
#    notch = find_fmax_noise(recording_cmr)
#    recording_cmr = st.preprocessing.notch_filter(recording_cmr, notch, q=100)
#    if len(dig_intan) > 1:
#        print(dig_intan[1].times)
#        stim_triggers = [int(d) for d in dig_intan[1].times.magnitude[::2] * recording.get_sampling_frequency()]
#        recording_rm = st.preprocessing.remove_artifacts(recording_cmr, np.array(stim_triggers), ms_before=5, ms_after=5)
#    else:
#        recording_rm = recording_cmr
#else:
#    recording_rm = recording_cmr

In [ ]:
fs = 30000
fs_lfp = 1000
start_time = 100
end_time = 150

In [ ]:
# choose tetrode
tetrode = 4
channel_ids = tetrode*4 + np.arange(4)
print(channel_ids)

In [ ]:
if check_hp:
    traces = recording_rm.get_traces(start_frame=start_time*fs, end_frame=end_time*fs)
    traces_f = recording_f.get_traces(start_frame=start_time*fs, end_frame=end_time*fs)
    timestamps = np.arange(traces.shape[1])/recording.get_sampling_frequency() + start_time
    print(traces.shape, timestamps.shape)

In [ ]:
if check_hp:
    plt.figure()
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    for i, ch in enumerate(channel_ids):
        plt.plot(timestamps, traces[ch] + 600*i, lw=0.5, color=colors[i])
        plt.plot(timestamps, traces_f[ch] + 600*i, lw=0.5, color=colors[i], alpha=0.3)
        if intan:
            [plt.axvline(d, lw=0.3, alpha=0.2) for d in dig_intan[1].times[::2]]

In [ ]:
# compute lfp and psd
recording_lfp = st.preprocessing.bandpass_filter(
            recording, freq_min=1, freq_max=300)
recording_lfp = st.preprocessing.resample(
    recording_lfp, 1000)
fs_lfp = 1000


In [ ]:
if check_lfp:
    traces_lfp = recording_lfp.get_traces(start_frame=start_time*fs_lfp, end_frame=end_time*fs_lfp)
    timestamps_lfp = np.arange(traces_lfp.shape[1])/recording_lfp.get_sampling_frequency() + start_time
    f, p = ss.welch(traces_lfp, fs=fs_lfp, nperseg=2048)

In [ ]:
if check_lfp:
    plt.figure()
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    for i, ch in enumerate(channel_ids):
        plt.plot(timestamps_lfp, traces_lfp[ch] + 600*i, lw=0.5, color=colors[i])

In [ ]:
if check_lfp:
    plt.figure()
    f_idx_100 = np.where(f < 100)
    f_100 = f[f_idx_100]
    p_100 = np.squeeze(p[:, f_idx_100])
    _ = plt.semilogy(f_100, p_100[:7].T, 'r', alpha=0.5)
    _ = plt.semilogy(f_100, p_100[6:15].T, 'b', alpha=0.5)

